## RAG Engine + Live API (GenAI SDK)

### Grab existing RAG Engine corpus name

In [ ]:
import vertexai
from vertexai import rag

vertexai.init(project="andrewcooley-genai-tests", location="us-central1")

# Get RAG corpus for use with Live API
rag_corpus = rag.get_corpus(name=list(rag.list_corpora())[-1].name)

# Define the tools for the agent - types matter (i.e. pydantic)!
TOOLS = [
    {
        "retrieval": {
            "vertex_rag_store": {
                "rag_resources": [
                    {"rag_corpus": f"{rag_corpus.name}"}
                ]
            }
        }
    }
]

### Create client and asynchronous connection to the Live API model with the RAG Engine tool

In [2]:
import asyncio
from google import genai
from google.genai import types

MODEL="gemini-2.0-flash-live-preview-04-09"

client = genai.Client(
  vertexai=True,
  project="andrewcooley-genai-tests",
  location="us-central1"
)

config = {"response_modalities": ["TEXT"], "tools": TOOLS}

async with client.aio.live.connect(model=MODEL, config=config) as session:

  prompt = "What is the pet policy for the Grand Horizon Resort & Hotel?"

  await session.send_client_content(turns={"parts": [{"text": prompt}]})

  async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text)

                model_turn = chunk.server_content.model_turn
                if model_turn:
                    for part in model_turn.parts:
                        if part.executable_code is not None:
                          print(part.executable_code.code)

                        if part.code_execution_result is not None:
                          print(part.code_execution_result.output)


The
 Grand Horizon Resort & Hotel welcomes well-behaved domestic pets (dogs and cats only)
 under certain conditions. A non-refundable pet fee of a specified amount will be charged to your
 room account. There may be a size or weight limit for pets. A maximum of a specified number of pets are allowed per guest room. All pets must be declared at
 the time of reservation and registered at check-in. Pets must be kept on a leash or in a carrier at all times when outside the guest room and in
 public areas of the hotel. Pets are not permitted in food and beverage outlets, pool areas, fitness centers, or other designated non-pet areas, except for service animals. Pets must not be left unattended in guest rooms for extended periods.
 For the safety and comfort of staff and your pet, housekeeping services will only be provided if the pet is not present in the room or is securely crated. Guests are responsible for cleaning up after their pets on hotel grounds and in designated pet relief areas.